## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-20-17-52-58 +0000,nypost,This man owns the world’s largest collection o...,https://nypost.com/2025/12/20/world-news/this-...
1,2025-12-20-17-46-07 +0000,nypost,Ghislaine Maxwell snapped in dozens of embarra...,https://nypost.com/2025/12/20/us-news/ghislain...
2,2025-12-20-17-36-45 +0000,nypost,"Brown, MIT shooter was a loner in Portugal for...",https://nypost.com/2025/12/20/us-news/shooter-...
3,2025-12-20-17-34-19 +0000,nyt,Koch Political Operation Spent Nearly $550 Mil...,https://www.nytimes.com/2025/12/20/us/politics...
4,2025-12-20-17-32-51 +0000,nypost,Will NYC have a white Christmas this year? For...,https://nypost.com/2025/12/20/us-news/will-nyc...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2383/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
78,trump,30
12,epstein,27
51,files,19
84,new,14
260,he,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
214,2025-12-19-21-06-58 +0000,nypost,DOJ releases thousands of Jeffrey Epstein case...,https://nypost.com/2025/12/19/us-news/doj-rele...,107
108,2025-12-20-04-35-07 +0000,nyt,Epstein Files Timeline: How the Trump Administ...,https://www.nytimes.com/2025/12/19/us/politics...,93
157,2025-12-19-23-51-09 +0000,nyt,Trump Is Rarely Mentioned in New Epstein Files,https://www.nytimes.com/2025/12/19/us/trump-ep...,92
110,2025-12-20-04-24-00 +0000,wsj,Days after President Trump moved to block stat...,https://www.wsj.com/articles/new-york-signs-ai...,87
141,2025-12-20-01-20-00 +0000,wsj,The Justice Department has released files rela...,https://www.wsj.com/politics/policy/justice-de...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
214,107,2025-12-19-21-06-58 +0000,nypost,DOJ releases thousands of Jeffrey Epstein case...,https://nypost.com/2025/12/19/us-news/doj-rele...
159,67,2025-12-19-23-43-00 +0000,wsj,"Rep. Elise Stefanik (R., N.Y.) said she would ...",https://www.wsj.com/politics/elections/elise-s...
61,53,2025-12-20-12-54-10 +0000,nypost,Inside Nick Reiner’s sheltered Hollywood upbri...,https://nypost.com/2025/12/20/us-news/inside-n...
10,42,2025-12-20-17-07-47 +0000,nypost,Jordan reveals air force joined US strikes on ...,https://nypost.com/2025/12/20/world-news/jorda...
188,41,2025-12-19-22-21-47 +0000,nypost,Elon Musk wins appeal to restore $56B Tesla pa...,https://nypost.com/2025/12/19/business/elon-mu...
174,38,2025-12-19-22-56-48 +0000,nypost,Rubio warns Ukraine war won’t end in surrender...,https://nypost.com/2025/12/19/world-news/rubio...
145,37,2025-12-20-01-00-09 +0000,nyt,How a Reddit Post Helped the Police Identify a...,https://www.nytimes.com/2025/12/19/us/brown-mi...
105,35,2025-12-20-05-47-00 +0000,wsj,President Trump traveled Friday to North Carol...,https://www.wsj.com/politics/policy/trump-tout...
4,33,2025-12-20-17-32-51 +0000,nypost,Will NYC have a white Christmas this year? For...,https://nypost.com/2025/12/20/us-news/will-nyc...
13,31,2025-12-20-17-02-00 +0000,wsj,Industry titans and Trump allies are weighing ...,https://www.wsj.com/politics/policy/the-fight-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
